## Oracle Docker Image Shared Functions
The following are a selection of shared functions used by various Juypter Notebooks that whilst useful don't need to included in the main notebooks.

In [11]:
import docker
import os
import tarfile
from prettytable import PrettyTable
from IPython.display import HTML, display, Markdown
import humanize
import re

In [12]:
def list_images(client):
    from prettytable import PrettyTable
    table = PrettyTable(["Tag", "id", "Size"])
    check_for_tags = lambda i: i.tags[0] if len(i.tags) != 0 else ''
    for image in client.images.list():
        table.add_row(
            [check_for_tags(image), image.short_id.replace('sha256:', ''), humanize.naturalsize(image.attrs['Size'])])
    display(HTML(table.get_html_string()))

In [13]:
def list_containers(client):
    table = PrettyTable(["Name", "Short id", "Status", "IP Address"])
    table.align["Name"] = "l"
    for container in client.containers.list(all=True):
        table.add_row([container.name, container.short_id, container.status, container.attrs['NetworkSettings']['IPAddress']])
        # for key, value in container.attrs.iteritems():
        #     print(key, value)
    display(HTML(table.get_html_string()))

In [14]:
def copy_to(src, dest, container):
    
    
    os.chdir(os.path.dirname(src))
    srcname = os.path.basename(src)
    tar = tarfile.open(src + '.tar', mode='w')
    try:
        tar.add(srcname)
    finally:
        tar.close()

    tar_data = open(src + '.tar', 'rb').read()
    container.put_archive(path=os.path.dirname(dest), data=tar_data)
#     os.remove(src)
    os.remove(src + '.tar')

In [15]:
def create_and_run_script(script, script_local_path, script_target_path,  command, target):
    
    copy_to(script, script_local_path, target.name+":"+script_target_path)

    print(command)
    target.start()
    output = target.exec_run(command,
                              stderr=True,
                              stdout=True,
                              stream=True
                              )
    for line in output[1]:
        o_line = line.decode('UTF-8')
        print(o_line.strip())

In [16]:
def containter_exec(target, target_command, show_output=True, stream_val=True):
    
    target.start()
    output = target.exec_run(target_command,
                                  stderr=True,
                                  stdout=True,
                                  stream=stream_val
                                  )
    if show_output:
        for line in output[1]:
            o_line = line.decode('UTF-8')
            print(o_line.strip())

In [17]:
def containter_root_exec(target, target_command, show_output=True, stream_val=True):
    
    target.start()
    output = target.exec_run(target_command,
                                  stderr=True,
                                  stdout=True,
                                  stream=stream_val,
                                  user='root',
                                  workdir='/'
                                  )
    if show_output:
        for line in output[1]:
            o_line = line.decode('UTF-8')
            print(o_line.strip())